In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AF9FdABh-vEUkXKZdebstr1iXzFp8aZ1SDCjOwZYHekaGmz0AHo8Js
Mounted at /content/drive


In [2]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt

import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
type(trainset.targets)
type(trainset.data)

numpy.ndarray

In [5]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

fg1,fg2,fg3 = 0,1,2

In [6]:
is_fg = [np.where(np.array(trainset.targets)==fg1)[0] , np.where(np.array(trainset.targets)==fg2)[0], np.where(np.array(trainset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   29    30    35 ... 49987 49991 49995] (15000,)


In [7]:
trainset.data = trainset.data[is_fg]
trainset.data.shape

(15000, 32, 32, 3)

In [8]:
trainset.targets = np.array(trainset.targets)[is_fg]
trainset.targets.shape

(15000,)

In [9]:
is_fg = [np.where(np.array(testset.targets)==fg1)[0] , np.where(np.array(testset.targets)==fg2)[0], np.where(np.array(testset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   3   10   21 ... 9970 9982 9989] (3000,)


In [10]:
testset.data = testset.data[is_fg]
testset.data.shape

(3000, 32, 32, 3)

In [11]:
testset.targets = np.array(testset.targets)[is_fg]
testset.targets.shape

(3000,)

In [12]:
np.unique(np.array(testset.targets)) , np.unique(np.array(trainset.targets))

(array([0, 1, 2]), array([0, 1, 2]))

In [13]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=0)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=0)
    self.conv3 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, padding=0)
    self.fc1 = nn.Linear(320, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 10)
    self.fc4 = nn.Linear(10,3)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    # print(x.shape)
    x = (F.relu(self.conv3(x)))
    x =  x.view(x.size(0), -1)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [19]:
what_net = CNN()#.double()
what_net = what_net.to("cuda")

In [20]:
what_net

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [21]:
import torch.optim as optim
criterion_what = nn.CrossEntropyLoss()
optimizer_what = optim.SGD(what_net.parameters(), lr=0.01, momentum=0.9)

In [22]:
acti = []
loss_curi = []
epochs = 1000
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_what.zero_grad()

        # forward + backward + optimize
        outputs = what_net(inputs)
        loss = criterion_what(outputs, labels)
        loss.backward()
        optimizer_what.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.005):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 1.108
[2,    50] loss: 1.098
[3,    50] loss: 1.098
[4,    50] loss: 1.097
[5,    50] loss: 1.094
[6,    50] loss: 1.079
[7,    50] loss: 1.007
[8,    50] loss: 0.960
[9,    50] loss: 0.935
[10,    50] loss: 0.896
[11,    50] loss: 0.799
[12,    50] loss: 0.649
[13,    50] loss: 0.569
[14,    50] loss: 0.566
[15,    50] loss: 0.538
[16,    50] loss: 0.510
[17,    50] loss: 0.493
[18,    50] loss: 0.473
[19,    50] loss: 0.488
[20,    50] loss: 0.467
[21,    50] loss: 0.438
[22,    50] loss: 0.432
[23,    50] loss: 0.435
[24,    50] loss: 0.406
[25,    50] loss: 0.390
[26,    50] loss: 0.390
[27,    50] loss: 0.376
[28,    50] loss: 0.355
[29,    50] loss: 0.341
[30,    50] loss: 0.327
[31,    50] loss: 0.312
[32,    50] loss: 0.312
[33,    50] loss: 0.295
[34,    50] loss: 0.284
[35,    50] loss: 0.272
[36,    50] loss: 0.272
[37,    50] loss: 0.256
[38,    50] loss: 0.240
[39,    50] loss: 0.223
[40,    50] loss: 0.230
[41,    50] loss: 0.218
[42,    50] loss: 0.191
[

In [23]:
torch.save(what_net.state_dict(),"/content/drive/My Drive/Research/Cheating_data/Classify_net_weights/classify_net_3layer_cnn_6_12_20"+".pt")

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = what_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 15000 train images: 100 %
15000 15000


In [25]:
correct = 0
total = 0
out = []
pred = []
what_net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= what_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 85 %
3000 2579
